<a href="https://colab.research.google.com/github/sophiascarano/hybrid_quantum_classical_KiUNet/blob/main/messy_drafts/helpful_finds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

     |████████████████████████████████| 6.0MB 8.0MB/s 
     |████████████████████████████████| 18.0MB 210kB/s 
     |████████████████████████████████| 235kB 41.8MB/s 
     |████████████████████████████████| 215kB 41.3MB/s 
     |████████████████████████████████| 2.1MB 27.6MB/s 
     |████████████████████████████████| 1.0MB 31.4MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 194kB 49.0MB/s 
     |████████████████████████████████| 112kB 49.2MB/s 
     |████████████████████████████████| 614kB 15.0MB/s 
     |████████████████████████████████| 3.2MB 30.2MB/s 
     |████████████████████████████████| 6.3MB 27.6MB/s 
  Created wheel for qiskit: filename=qiskit-0.26.0-cp37-none-any.whl size=10489 sha256=63d0ad15b19059019d2bf8ded213c5ce7b52409c992a563c83ffdda0c0d84987
  Stored in directory: /root/.cache/pip/wheels/77/b5/a2/e88ddacf664ca9d2d30275664482d128c3e28b1fd554124a9d
  Created wheel for python-constraint: filename=python_constraint-1.4.0-

In [ ]:
pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

  Cloning https://github.com/qiskit-community/qiskit-textbook.git to /tmp/pip-req-build-zfcz4vhz
  Running command git clone -q https://github.com/qiskit-community/qiskit-textbook.git /tmp/pip-req-build-zfcz4vhz
  Created wheel for qiskit-textbook: filename=qiskit_textbook-0.1.0-cp37-none-any.whl size=16953 sha256=534d38fbc852f61922ab3a05d3cbd28d4254e32f61678192daf02b8b3007adab
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ftv2422/wheels/0c/57/26/1c12886a219af04a38ca1e674fdeeff40b8a5963326ba3c71a
Successfully built qiskit-textbook


In [ ]:

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
# inherit from this base class
from tensorflow.keras.layers import Layer

In [ ]:
from qiskit.aqua.operators import Z, Y, X
from qiskit.aqua.operators import StateFn
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

QUBITS = 4
operatorZ = Z ^ Z ^ Z ^ Z
operatorX = X ^ X ^ X ^ X
operatorY = Y ^ Y ^ Y ^ Y

def quantum_layer(initial_parameters):
    # expecting parameters to be a numpy array
    quantumRegister = QuantumRegister(QUBITS)
    quantumCircuit = QuantumCircuit(quantumRegister)
    
    quantumCircuit.h(range(4))

    for i in range(len(initial_parameters)):
      quantumCircuit.ry(initial_parameters[i] * np.pi, i)
   
    psi = StateFn(quantumCircuit)
    
    # two ways of doing the same thing
    expectationX = (~psi @ operatorX @ psi).eval()
    expectationZ = psi.adjoint().compose(operatorZ).compose(psi).eval().real
    expectationY = (~psi @ operatorY @ psi).eval()
    
    expectationZ = np.abs(np.real(expectationZ))
    expectations = [expectationX, expectationY, expectationZ, 
                    expectationX + expectationY + expectationZ] 

    return np.array(expectations)

In [ ]:
class Linear(Layer):
    def __init__(self, batch_size=64, units=4,**kwargs):
        super(Linear, self).__init__(**kwargs)
        

    def get_config(self):
        config = super(Linear, self).get_config()
        return config

    def call(self, inputs):
        
        if(tf.executing_eagerly()):
            final_output = []
            for i in range(inputs.shape[0]):
              pred = quantum_layer(inputs[i].numpy())
              final_output.append(list(pred))
            return tf.convert_to_tensor(final_output)
        return inputs


In [ ]:
x_train=np.array([[1,2],[3,4],[3,4]])
#tf.config.run_functions_eagerly(True)
#sess = tf.Session()
inputs = tf.keras.layers.Input(shape=(2,))
output_1 = tf.keras.layers.Dense(20, kernel_initializer='glorot_normal')(inputs)
layer = Linear()(output_1)
output_2 = tf.keras.layers.Dense(2, activation='linear',kernel_initializer='glorot_normal')(layer)
model2 = tf.keras.models.Model(inputs=inputs, outputs=output_2)

model2.compile(optimizer='adam',loss='mean_squared_error', run_eagerly=True,)
model2.fit(x_train,x_train,epochs=10,validation_split=0.1,shuffle=False)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


CircuitError: ignored

In [ ]:
v = tf.Variable(0.0)
#a = v + 1
a = v.read_value().numpy() + 1
b = tf.Variable(a, dtype='float32')
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [ ]:
v[0]

In [ ]:
v + 1

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [ ]:
b

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>

In [ ]:
v = tf.Variable(0.0)
#a = v + 1
v = v.read_value().numpy() + 1
v = tf.Variable(v, dtype='float32')
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>

https://stackoverflow.com/questions/56657993/how-to-create-a-keras-layer-with-a-custom-gradient-in-tf2-0

In [ ]:
#Note that in the function you should treat x and dy as Tensors and not numpy 
# arrays (i.e. perform tensor operations)
@tf.custom_gradient
def custom_op(x):
    result = ... # do forward computation
    def custom_grad(dy):
        grad = ... # compute gradient
        return grad
    return result, custom_grad

In [ ]:
from tensorflow.python.framework import ops

In [ ]:
def my_sigmoid(x, name=None):
  """Computes sigmoid of `x` element-wise.
  Specifically, `y = 1 / (1 + exp(-x))`.
  Args:
    x: A Tensor with type `float16`, `float32`, `float64`, `complex64`,
      or `complex128`.
    name: A name for the operation (optional).
  Returns:
    A Tensor with the same type as `x`.
  @compatibility(numpy)
  Equivalent to np.scipy.special.expit
  @end_compatibility
  """
  with ops.name_scope(name, "Sigmoid", [x]) as name:
    x = ops.convert_to_tensor(x, name="x")
    return gen_math_ops.sigmoid(x, name=name)

In [ ]:
tf.compat.v1.enable_eager_execution() 
def my_sigmoid(x, name=None):
  """Computes sigmoid of `x` element-wise.
  Specifically, `y = 1 / (1 + exp(-x))`.
  Args:
    x: A Tensor with type `float16`, `float32`, `float64`, `complex64`,
      or `complex128`.
    name: A name for the operation (optional).
  Returns:
    A Tensor with the same type as `x`.
  @compatibility(numpy)
  Equivalent to np.scipy.special.expit
  @end_compatibility
  """
  tf.enable_eager_execution(
  func = np.exp(x.numpy().decode("utf-8") )
  return tf.convert_to_tensor(func)

In [ ]:
tf.compat.v1.enable_eager_execution() 
def my_sigmoid(x, name=None):
  """Computes sigmoid of `x` element-wise.
  Specifically, `y = 1 / (1 + exp(-x))`.
  Args:
    x: A Tensor with type `float16`, `float32`, `float64`, `complex64`,
      or `complex128`.
    name: A name for the operation (optional).
  Returns:
    A Tensor with the same type as `x`.
  @compatibility(numpy)
  Equivalent to np.scipy.special.expit
  @end_compatibility
  """
  @compatibility(numpy)
  #tf.compat.v1.enable_eager_execution() 
  func = np.exp(x.numpy().decode("utf-8") )
  return tf.convert_to_tensor(func)

In [ ]:
import numpy as np
import random
import tensorflow as tf
import math
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation
from keras import regularizers
from keras import backend as K
#from keras.backend import tf
import tensorflow as tf
from keras import initializers


@tf.custom_gradient
@tf.compatibility(numpy)
def custom_activation(x):
    result = my_sigmoid(x)
    def grad(dy):
        grad=0;
        return dy * grad

    return result, grad



class CustomLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(CustomLayer, self).__init__()

    def call(self, x):
        return custom_activation(x)

In [ ]:
x_train=np.array([[1,2],[3,4],[3,4]])

In [ ]:
tf.compat.v1.enable_eager_execution() 

x_train=np.array([[1,2],[3,4],[3,4]])
#tf.config.run_functions_eagerly(True)
#sess = tf.Session()
inputs = tf.keras.layers.Input(shape=(2,))
output_1 = tf.keras.layers.Dense(20, kernel_initializer='glorot_normal')(inputs)
layer = CustomLayer()(output_1)
output_2 = tf.keras.layers.Dense(2, activation='linear',kernel_initializer='glorot_normal')(layer)
model2 = tf.keras.models.Model(inputs=inputs, outputs=output_2)

model2.compile(optimizer='adam',loss='mean_squared_error', run_eagerly=True,)
model2.fit(x_train,x_train,epochs=10,validation_split=0.1,shuffle=False)

AttributeError: ignored